<div class="alert alert-block alert-info">
<h1 style="text-align:center;font-family:avenir;">ICA first</h1>
Lou Planchamp & Victor Férat
</div>

In [ ]:
%matplotlib inline


# Multivariate decompositions: Independent component analysis of fMRI


This example is meant to demonstrate nilearn as a low-level tools used to
combine feature extraction with a multivariate decomposition algorithm
for movie-watching.

This example is a toy. To apply ICA to fmri timeseries data, it is advised
to look at the example
`sphx_glr_auto_examples_03_connectivity_plot_compare_decomposition.py`.

The example here applies the scikit-learn :term:`ICA` to movie watching
timeseries data. Note that following the code in the example, any unsupervised
decomposition model, or other latent-factor models, can be applied to
the data, as the scikit-learn API enables to exchange them as almost
black box (though the relevant parameter for brain maps might no longer
be given by a call to fit_transform).

.. include:: ../../../examples/masker_note.rst


Load movie watching dataset



In [ ]:
from os.path import join
from bids import BIDSLayout
from bids.tests import get_test_data_path
bids_folder = '/home/carole/Bureau/BIDS_fMRIprep'
layout = BIDSLayout(bids_folder, derivatives=True)

In [ ]:
layout

In [ ]:
func_filenames = layout.get(task='rest', run=[1,2], space='MNIPediatricAsym', desc='preproc', extension='nii.gz', return_type='filename')

In [ ]:
func_filenames

In [ ]:
,02,05,08 subject=[1], 

In [ ]:
confounds = layout.get(task='rest', subject=[1,2,5,8], run=[1,2], desc='confounds', extension='tsv', return_type='filename')

Preprocess



In [ ]:
func_filenames[0]

In [ ]:
import nilearn
img = nilearn.image.load_img(func_filenames[0])

In [ ]:
import pandas as pd
confounds = [pd.read_table(confound).fillna(0) for confound in confounds]

In [ ]:
r = canica.transform(func_filenames, confounds=confounds)

In [ ]:
len(confounds)

In [ ]:
len(r)

In [ ]:
 nilearn.image.load_img(func_filenames[0]).get_fdata().shape

In [ ]:
corr_matrix = np.corrcoef(r[0].T) - np.corrcoef(r[2].T)

In [ ]:
import seaborn as sns

corr_matrix = np.corrcoef(r[0].T)
plt.figure(figsize=(8,8))

sns.heatmap(np.abs(corr_matrix)*100,
            vmin=0, vmax=100, cmap="YlGnBu",
            annot=True,
            fmt=".0f"
            )

In [ ]:
import seaborn as sns

corr_matrix = np.corrcoef(r[0].T) - np.corrcoef(r[2].T)
plt.figure(figsize=(8,8))

sns.heatmap(corr_matrix*100,
            vmin=-100, vmax=100, cmap="RdBu",
            annot=True,
            fmt=".0f"
            )

In [ ]:
func_filenames[0]

In [ ]:
from nilearn.decomposition import CanICA

canica = CanICA(n_components=20,
                memory="nilearn_cache", memory_level=2,
                verbose=10,
                mask_strategy='whole-brain-template',
                random_state=0)
canica.fit(func_filenames)

# Retrieve the independent components in brain space. Directly
# accessible through attribute `components_img_`.
canica_components_img = canica.components_img_
# components_img is a Nifti Image object, and can be saved to a file with
# the following line:
canica_components_img.to_filename('canica_resting_state.nii.gz')

In [ ]:
from nilearn.plotting import plot_prob_atlas

# Plot all ICA components together
plot_prob_atlas(canica_components_img, title='All ICA components')

In [ ]:
from nilearn.plotting import plot_prob_atlas

# Plot all ICA components together
plot_prob_atlas(canica_components_img, view_type = 'filled_contours',dim=0,display_mode='z',
                         cut_coords=5, colorbar=True, title='All ICA components')

In [ ]:
from nilearn import plotting
plotting.view_img(canica_components_img.slicer[:,:,:,10])


In [ ]:
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show
import matplotlib.pyplot as plt

#plt.rcParams['axes.facecolor'] = 'white'
n_columns = 2
n_ica = 20
nrows = int(n_ica / n_columns)
fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))

for i, cur_img in enumerate(iter_img(canica_components_img)):
    plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                  cut_coords=3, 
                  colorbar=False, 
                  axes=axes[int(i / 2), int(i % 2)])

Apply ICA



In [ ]:
from sklearn.decomposition import FastICA
n_components = 10
ica = FastICA(n_components=n_components, random_state=42)
components_masked = ica.fit_transform(data_masked.T).T

# Normalize estimated components, for thresholding to make sense
components_masked -= components_masked.mean(axis=0)
components_masked /= components_masked.std(axis=0)
# Threshold
import numpy as np
components_masked[np.abs(components_masked) < .8] = 0

# Now invert the masking operation, going back to a full 3D
# representation
component_img = masker.inverse_transform(components_masked)

Visualize the results



In [ ]:
# Show some interesting components
from nilearn import image
from nilearn.plotting import plot_stat_map, show

# Use the mean as a background
mean_img = image.mean_img(func_filename)

plot_stat_map(image.index_img(component_img, 0), mean_img)

plot_stat_map(image.index_img(component_img, 1), mean_img)

plot_stat_map(image.index_img(component_img, 2), mean_img)

plot_stat_map(image.index_img(component_img, 3), mean_img)

show()

In [ ]:
for subject in layout.get_subjects():
    if subject not in ['07']:
        for session in layout.get_sessions(subject=subject):
            f = layout.get(task='rest', subject=subject, session=session,
                           space='MNIPediatricAsym', desc='preproc', extension='nii.gz', 
                           return_type='filename')

            print(subject, session, f)

            canica = CanICA(n_components=20,
                            memory="nilearn_cache", memory_level=2,
                            verbose=1,
                            mask_strategy='whole-brain-template',
                            smoothing_fwhm=10,
                            random_state=0)
            canica.fit(f)


            if not os.path.isdir('ICA'):
                os.mkdir('ICA')
            if not os.path.isdir('ICA/individual'): 
                os.mkdir('ICA/individual')
            sub_folder = f'ICA/individual/{subject}'
            if not os.path.isdir(sub_folder): 
                os.mkdir(sub_folder)

            sub_path = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_ica.nii')

            # Retrieve the independent components in brain space. Directly ccessible through attribute `components_img_`.
            canica_components_img = canica.components_img_
            # components_img is a Nifti Image object, and can be saved to a file with he following line:
            canica_components_img.to_filename(sub_path)
            
            sub_path_fig1 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig1_ica.png')

            plot_prob_atlas(canica_components_img, output_file=sub_path_fig1)
            
            n_columns = 2
            n_ica = 20
            nrows = int(n_ica / n_columns)
            fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))
            
            sub_path_fig2 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig2_ica.png')

            for i, cur_img in enumerate(iter_img(canica_components_img)):
                plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                              output_file=sub_path_fig2,
                              cut_coords=3, 
                              colorbar=False, 
                              axes=axes[int(i / 2), int(i % 2)])



In [ ]:
for subject in layout.get_subjects():
    if subject not in ['07']:
        for session in layout.get_sessions(subject=subject):
            f = layout.get(task='rest', subject=['07'], session=session,
                           space='MNIPediatricAsym', desc='preproc', extension='nii.gz', 
                           return_type='filename')

            print(subject, session, f)

            canica = CanICA(n_components=20,
                            memory="nilearn_cache", memory_level=2,
                            verbose=1,
                            mask_strategy='whole-brain-template',
                            smoothing_fwhm=10,
                            random_state=0)
            canica.fit(f)


            if not os.path.isdir('ICA'):
                os.mkdir('ICA')
            if not os.path.isdir('ICA/individual'): 
                os.mkdir('ICA/individual')
            sub_folder = f'ICA/individual/{subject}'
            if not os.path.isdir(sub_folder): 
                os.mkdir(sub_folder)

            sub_path = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_ica.nii')

            # Retrieve the independent components in brain space. Directly ccessible through attribute `components_img_`.
            canica_components_img = canica.components_img_
            # components_img is a Nifti Image object, and can be saved to a file with he following line:
            canica_components_img.to_filename(sub_path)
            
            sub_path_fig1 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig1_ica.png')

            plot_prob_atlas(canica_components_img, output_file=sub_path_fig1)
            
            n_columns = 2
            n_ica = 20
            nrows = int(n_ica / n_columns)
            fig, axes = plt.subplots(nrows=nrows, ncols=n_columns, figsize=(20,60))
            
            sub_path_fig2 = os.path.join(sub_folder, f'sub-{subject}_ses-{session}_fig2_ica.png')

            for i, cur_img in enumerate(iter_img(canica_components_img)):
                plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                              output_file=sub_path_fig2,
                              cut_coords=3, 
                              colorbar=False, 
                              axes=axes[int(i / 2), int(i % 2)])



In [ ]:

wiw='/home/carole/Bureau/nilearn/ICA/individual/08/sub-08_ses-11_ica.nii'
plot_prob_atlas(wiw)